<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Найти-аэропорт-с-минимальной-задержкой-вылета" data-toc-modified-id="Найти-аэропорт-с-минимальной-задержкой-вылета-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Найти аэропорт с минимальной задержкой вылета</a></span></li><li><span><a href="#Самая-пунктуальная-авиакомпания-на-прилет-в-Los-Angeles-International-Airport" data-toc-modified-id="Самая-пунктуальная-авиакомпания-на-прилет-в-Los-Angeles-International-Airport-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport</a></span></li><li><span><a href="#Аэропорт-с-самой-большой-долей-руления-на-1-самолетовылет" data-toc-modified-id="Аэропорт-с-самой-большой-долей-руления-на-1-самолетовылет-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Аэропорт с самой большой долей руления на 1 самолетовылет</a></span></li><li><span><a href="#Построить-модель-которая-будет-выбирать-топ-3-аэропорта-прилета-(вероятность-опоздания-минимальная-–-RMSE-метрика),-в-зависимости-от-аэропорта-вылета" data-toc-modified-id="Построить-модель-которая-будет-выбирать-топ-3-аэропорта-прилета-(вероятность-опоздания-минимальная-–-RMSE-метрика),-в-зависимости-от-аэропорта-вылета-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Выбор-параметров-и-получение-результата" data-toc-modified-id="Выбор-параметров-и-получение-результата-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Выбор параметров и получение результата</a></span></li></ul></li></ul></div>

[Данные - https://www.kaggle.com/usdot/flight-delays  ](https://www.kaggle.com/usdot/flight-delays) 

Описание - Сводная информация о количестве своевременных, задержанных, отмененных и измененных рейсов в США за 2015 год (5.8 млн записей)
Задачи:
1	Найти аэропорт с минимальной задержкой вылета 
2	Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport
3	Аэропорт с самой большой долей руления на 1 самолетовылет
4	Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика),  в зависимости от аэропорта вылета  

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
%%time
airlines= pd.read_csv('data/airlines.csv')
airports= pd.read_csv('data/airports.csv')
flights= pd.read_csv('data/flights.csv',dtype={'ORIGIN_AIRPORT':str,'DESTINATION_AIRPORT':str})

In [ ]:
%%time
flights1=flights.merge(airports[['IATA_CODE','AIRPORT']], how='inner', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE').drop (['IATA_CODE'], axis = 1)

In [ ]:
%%time
flights1=flights1.merge(airports[['IATA_CODE','AIRPORT']], how='inner', left_on='DESTINATION_AIRPORT', right_on='IATA_CODE').drop (['IATA_CODE',], axis = 1)

In [ ]:
%%time
flights1=flights1.merge(airlines, how='inner', left_on='AIRLINE', right_on='IATA_CODE').drop (['IATA_CODE'], axis = 1)

In [ ]:
%%time
flights_real= flights1[~flights1.DIVERTED.isin([1])& ~flights1.CANCELLED.isin([1])].copy()

In [ ]:
flights_real.info()

In [ ]:
#flights_real.nunique()

# 	Найти аэропорт с минимальной задержкой вылета

In [ ]:
flights_dep_delay=flights_real.query("DEPARTURE_DELAY>=0").groupby(['ORIGIN_AIRPORT','AIRPORT_x' ])['DEPARTURE_DELAY'].median().sort_values().reset_index()
flights_dep_delay.head(10)

In [ ]:
flights_real.query("DEPARTURE_DELAY>=0").boxplot('DEPARTURE_DELAY',figsize = (14, 14))

Использована медиана, а не среднее, так как большое количество выбросов по полю DEPARTURE_DELAY.

#  Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport 

In [ ]:
flights_ariv_delay=flights_real.query("ARRIVAL_DELAY>=0 & DESTINATION_AIRPORT=='LAX'").groupby(['AIRLINE_x','AIRLINE_y' ])['ARRIVAL_DELAY'].median().sort_values().reset_index()
flights_ariv_delay.head(1)

In [ ]:
flights_real.query("ARRIVAL_DELAY>=0 & DESTINATION_AIRPORT=='LAX'").boxplot('ARRIVAL_DELAY',figsize = (14, 14))

Использована медиана, а не среднее, так как большое количество выбросов по полю ARRIVAL_DELAY.

# Аэропорт с самой большой долей руления на 1 самолетовылет

In [ ]:
flights_TAXI_OUT= (flights_real.groupby(['ORIGIN_AIRPORT','AIRPORT_x']).TAXI_OUT.sum()/flights_real.groupby(['ORIGIN_AIRPORT','AIRPORT_x']).YEAR.count()).sort_values(ascending=False).reset_index()
flights_TAXI_OUT.head(1)

# Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета

## Подготовка данных 

In [ ]:
def make_date(df):
 return dt.date(df.YEAR,df.MONTH,df.DAY)

In [ ]:
%%time
flights_real['dDate']=flights_real.apply(make_date, axis=1)

In [ ]:
%%time
flight_grouped = flights_real.groupby(['dDate','DESTINATION_AIRPORT'])['ARRIVAL_DELAY'].sum().reset_index()
flight_grouped['dDate'] = pd.to_datetime(flight_grouped['dDate'])

In [ ]:
def make_features(data, max_lag, rolling_mean_size):
    data['YEAR'] =data.dDate.dt.year
    data['MONTH'] =data.dDate.dt.month
    data['DAY'] =data.dDate.dt.day
    data['DAYOFWEEK'] =data.dDate.dt.dayofweek
    data['is_weekend'] = data.DAYOFWEEK.isin([5,6])*1
    for lag in range (1, max_lag+1):
        data['ARRIVAL_DELAY_{}'.format(lag)] = data['ARRIVAL_DELAY'].shift(lag)     

In [ ]:
%%time
where_to_go = []
for dest in flight_grouped[flight_grouped.ARRIVAL_DELAY>=0].DESTINATION_AIRPORT.unique():
    tempo = flight_grouped[(flight_grouped.DESTINATION_AIRPORT==dest)&(flight_grouped.ARRIVAL_DELAY>=0)][['dDate','ARRIVAL_DELAY']]
    tempo.columns = ['dDate','ARRIVAL_DELAY']
    tempo = pd.DataFrame(tempo)
    
    try:
    
        make_features(tempo,5,2)
        tempo.dropna(inplace=True)
        tempo.set_index('dDate',inplace=True)
           
        X_train,X_test,y_train, y_test = train_test_split(tempo.drop('ARRIVAL_DELAY', axis=1),tempo.ARRIVAL_DELAY, shuffle=False, test_size=0.1)

        model_lr = LinearRegression()
        model_lr.fit(X_train,y_train)

        y_predicted_lr = model_lr.predict(X_test)

        where_to_go.append([dest,y_test.mean(),np.sqrt(mean_squared_error(y_test, y_predicted_lr))])

    except Exception as e:
        print('Error', str(e))

In [ ]:
%%time
where_to_go = pd.DataFrame(where_to_go)
where_to_go.columns = ['DESTINATION_AIRPORT','MEAN_ARRIVAL_DELAY_IN_PAST','RMSE']

In [ ]:
where_to_go.head()

## Выбор параметров и получение результата

In [ ]:
%%time
start_airport = np.random.choice(flights['ORIGIN_AIRPORT'].unique()) 
where_to_go_from_start_airport = flights[flights.ORIGIN_AIRPORT==start_airport]['DESTINATION_AIRPORT'].unique()

where_to_go_from_start_airport = pd.DataFrame(where_to_go_from_start_airport)
where_to_go_from_start_airport.columns = ['DESTINATION_AIRPORT']

top3 = where_to_go_from_start_airport\
    .merge(where_to_go,on='DESTINATION_AIRPORT',how='inner')\
    .sort_values(by=['RMSE','MEAN_ARRIVAL_DELAY_IN_PAST'],ascending=[True, True])\
    .head(3)

print('Лучшие направления с аэропорта',start_airport)
print('')
print(top3)